In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [2]:
train = pd.read_csv("data/train_v9rqX0R.csv", low_memory=False)
test = pd.read_csv("data/test_AbJTz2l.csv", low_memory=False)
test['Item_Outlet_Sales'] = np.nan

In [3]:

data = pd.concat([train, test], ignore_index=True)

In [4]:
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({
    'LF': 'Low Fat', 'low fat': 'Low Fat', 'reg': 'Regular'
})

In [5]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            14204 non-null  object 
 1   Item_Weight                11765 non-null  float64
 2   Item_Fat_Content           14204 non-null  object 
 3   Item_Visibility            14204 non-null  float64
 4   Item_Type                  14204 non-null  object 
 5   Item_MRP                   14204 non-null  float64
 6   Outlet_Identifier          14204 non-null  object 
 7   Outlet_Establishment_Year  14204 non-null  int64  
 8   Outlet_Size                10188 non-null  object 
 9   Outlet_Location_Type       14204 non-null  object 
 10  Outlet_Type                14204 non-null  object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 1.3+ MB


In [7]:
data.isna().sum()

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
dtype: int64

In [8]:
# Fill missing Item_Weight with mean
data['Item_Weight'].fillna(data['Item_Weight'].median(), inplace=True)
data['Item_Weight'].isna().sum()


0

In [9]:
data['Price_per_kg'] = data['Item_MRP'] / data['Item_Weight']

In [10]:
data['Outlet_Size'].unique()

array(['Medium', nan, 'High', 'Small'], dtype=object)

In [11]:
data['Outlet_Type'].unique()

array(['Supermarket Type1', 'Supermarket Type2', 'Grocery Store',
       'Supermarket Type3'], dtype=object)

In [12]:
# Fill missing Outlet_Size using mode per Outlet_Type
data['Outlet_Size'].fillna(data.groupby('Outlet_Type')['Outlet_Size'].transform(lambda x: x.mode()[0]), inplace=True)
data['Outlet_Size'].isna().sum()

0

In [13]:
# Create New Feature: Outlet_Years
data['Outlet_Years'] = 2025 - data['Outlet_Establishment_Year']

In [14]:
data['Item_Type'].unique()

array(['Dairy', 'Soft Drinks', 'Meat', 'Fruits and Vegetables',
       'Household', 'Baking Goods', 'Snack Foods', 'Frozen Foods',
       'Breakfast', 'Health and Hygiene', 'Hard Drinks', 'Canned',
       'Breads', 'Starchy Foods', 'Others', 'Seafood'], dtype=object)

In [15]:
## make groups for item type
perishables = ['Dairy', 'Fruits and Vegetables', 'Meat', 'Frozen Foods', 'Seafood', 'Breads', 'Breakfast']
non_perishables = ['Canned', 'Baking Goods', 'Starchy Foods', 'Household', 'Health and Hygiene']
consumables = ['Snack Foods', 'Soft Drinks', 'Hard Drinks']
other = ['Others']

def group_item_type(x):
    if x in perishables:
        return 'Perishable'
    elif x in non_perishables:
        return 'Non-Perishable'
    elif x in consumables:
        return 'Consumables'
    else:
        return 'Others'

data['Item_Category_Grouped'] = data['Item_Type'].apply(group_item_type)



In [16]:
data['Item_Category_Grouped'].unique()

array(['Perishable', 'Consumables', 'Non-Perishable', 'Others'],
      dtype=object)

In [17]:
data['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular'], dtype=object)

In [18]:
data['Outlet_Location_Type'].unique()

array(['Tier 1', 'Tier 3', 'Tier 2'], dtype=object)

In [19]:

# Create New Feature: Item_Visibility_MeanRatio
data['Item_Visibility_MeanRatio'] = data['Item_Visibility'] / data.groupby('Item_Identifier')['Item_Visibility'].transform('mean')
data['Item_Visibility_MeanRatio'].replace([np.inf, -np.inf], 0, inplace=True)
data['Item_Visibility_MeanRatio'].fillna(0, inplace=True)

In [20]:
# Encode Categorical Variables
le = LabelEncoder()
categorical_cols = ['Item_Fat_Content', 'Outlet_Location_Type', 'Outlet_Size', 'Outlet_Type', 'Item_Category_Grouped']

for col in categorical_cols:
    data[col] = le.fit_transform(data[col])

In [21]:
data

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Price_per_kg,Outlet_Years,Item_Category_Grouped,Item_Visibility_MeanRatio
0,FDA15,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,1,0,1,3735.1380,26.861204,26,3,0.931078
1,DRC01,5.92,1,0.019278,Soft Drinks,48.2692,OUT018,2009,1,2,2,443.4228,8.153581,16,0,0.933420
2,FDN15,17.50,0,0.016760,Meat,141.6180,OUT049,1999,1,0,1,2097.2700,8.092457,26,3,0.960069
3,FDX07,19.20,1,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,2,2,0,732.3800,9.484115,27,3,0.000000
4,NCD19,8.93,0,0.000000,Household,53.8614,OUT013,1987,0,2,1,994.7052,6.031512,38,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14199,FDB58,10.50,1,0.013496,Snack Foods,141.3154,OUT046,1997,2,0,1,NaN,13.458610,28,0,0.874729
14200,FDD47,7.60,1,0.142991,Starchy Foods,169.1448,OUT018,2009,1,2,2,NaN,22.255895,16,1,0.878292
14201,NCO17,10.00,0,0.073529,Health and Hygiene,118.7440,OUT045,2002,2,1,1,NaN,11.874400,23,1,1.162245
14202,FDJ26,15.30,1,0.000000,Canned,214.6218,OUT017,2007,2,1,1,NaN,14.027569,18,1,0.000000


In [22]:
# Final list of features
features = [
    'Item_Weight', 'Item_Visibility', 'Item_MRP',
    'Outlet_Years',
    'Item_Fat_Content', 'Outlet_Location_Type',
    'Outlet_Size', 'Outlet_Type', 'Item_Category_Grouped'
]

# features = [
#     'Item_Weight', 'Item_Visibility', 'Item_MRP',
#     'Outlet_Years',
#     'Item_Fat_Content', 'Outlet_Location_Type',
#     'Outlet_Size', 'Outlet_Type', 'Item_Type']

In [23]:
ID_columns = ['Item_Identifier', 'Outlet_Identifier']

In [24]:
train_df = data[~data['Item_Outlet_Sales'].isna()]
test_df = data[data['Item_Outlet_Sales'].isna()]

In [25]:
#### now further split train into train and validation

X = train_df[features]
y = train_df['Item_Outlet_Sales']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [26]:

models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.01),
    "ElasticNet": ElasticNet(alpha=0.01, l1_ratio=0.5),
    
    "Decision Tree": DecisionTreeRegressor(max_depth=10, random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),
    "Extra Trees": ExtraTreesRegressor(n_estimators=100, random_state=42),
    
    "AdaBoost": AdaBoostRegressor(n_estimators=100, learning_rate=0.1, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42),
    
    "K-Nearest Neighbors": KNeighborsRegressor(n_neighbors=5),
    "Support Vector Regressor": SVR(kernel='rbf', C=1.0, epsilon=0.2)
}

metrics = []
for name, model in models.items():
    model.fit(X_train, y_train)
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_val)
    
    rmse_train = np.sqrt(mean_squared_error(y_train, train_preds))
    r2_train = r2_score(y_train, train_preds)
    rmse_val = np.sqrt(mean_squared_error(y_val, val_preds))
    r2_val = r2_score(y_val, val_preds)
    
    metrics.append({
        'Model': name,
        'RMSE_Train': round(rmse_train, 2),
        'R2_Train': round(r2_train, 2),
        'RMSE_val': round(rmse_val, 2),
        'R2_val': round(r2_val, 2)
    })

# Convert to DataFrame
metrics_df = pd.DataFrame(metrics)

In [542]:
metrics_df

,Model,RMSE_Train,R2_Train,RMSE_val,R2_val
0,Linear Regression,1219.74,0.50,1142.94,0.52
1,Ridge Regression,1219.75,0.50,1142.77,0.52
2,Lasso Regression,1219.74,0.50,1142.93,0.52
3,ElasticNet,1221.29,0.50,1142.59,0.52
4,Decision Tree,959.41,0.69,1145.81,0.52
5,Random Forest,910.20,0.72,1040.39,0.60
6,Extra Trees,0.55,1.00,1122.46,0.54
7,AdaBoost,1135.18,0.56,1088.44,0.56
8,Gradient Boosting,948.99,0.70,1049.17,0.60
9,XGBoost,900.29,0.73,1055.07,0.59


In [543]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Define parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Instantiate the base model
rf = RandomForestRegressor(random_state=42)

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=30,  # number of combinations to try
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    random_state=42
)

# Fit on training set

# Combine X_train and X_val
X_full_train = pd.concat([X_train, X_val], axis=0)
y_full_train = pd.concat([y_train, y_val], axis=0)

random_search.fit(X_full_train, y_full_train)

# Best model and params
best_rf = random_search.best_estimator_
print("\nBest Parameters found:")
print(random_search.best_params_)

# Evaluate on validation set
train_preds = best_rf.predict(X_train)
val_preds = best_rf.predict(X_val)

rmse_train = np.sqrt(mean_squared_error(y_train, train_preds))
r2_train = r2_score(y_train, train_preds)
rmse_val = np.sqrt(mean_squared_error(y_val, val_preds))
r2_val = r2_score(y_val, val_preds)

print(f"\n Tuned Random Forest Performance:")
print(f"RMSE Train: {rmse_train:.2f}, R2 Train: {r2_train:.2f}")
print(f"RMSE Validation: {rmse_val:.2f}, R2 Validation: {r2_val:.2f}")


Fitting 5 folds for each of 30 candidates, totalling 150 fits


ValueError: 
All the 150 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Anconda_new\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Anconda_new\Lib\site-packages\sklearn\ensemble\_forest.py", line 345, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "d:\Anconda_new\Lib\site-packages\sklearn\base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anconda_new\Lib\site-packages\sklearn\utils\validation.py", line 1106, in check_X_y
    X = check_array(
        ^^^^^^^^^^^^
  File "d:\Anconda_new\L

In [ ]:
# Predict on test set using the best tuned model
final_test_preds = best_rf.predict(test_df[features])

# Attach predictions to test set
test_df['Item_Outlet_Sales'] = final_test_preds

# Create submission file
submission = test_df[['Item_Identifier', 'Outlet_Identifier', 'Item_Outlet_Sales']]
submission.to_csv("BigMart_Prediction_Submission.csv", index=False)


C:\Users\Nitro\AppData\Local\Temp\ipykernel_16200\1155128571.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Item_Outlet_Sales'] = final_test_preds


In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

param_grid_xgb = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.01, 0.1, 1],
    'reg_lambda': [1, 1.5, 2, 3]
}

xgb = XGBRegressor(random_state=42)

random_search_xgb = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid_xgb,
    n_iter=30,
    cv=5,
    scoring='neg_root_mean_squared_error',
    verbose=2,
    n_jobs=-1,
    random_state=42
)

random_search_xgb.fit(X_full_train, y_full_train)

best_xgb = random_search_xgb.best_estimator_
print("\nBest Parameters (XGBoost):")
print(random_search_xgb.best_params_)


Fitting 5 folds for each of 30 candidates, totalling 150 fits

Best Parameters (XGBoost):
{'subsample': 0.8, 'reg_lambda': 1, 'reg_alpha': 1, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 1.0}


In [ ]:
# Predict on the final test set using tuned XGBoost model
test_preds = best_xgb.predict(test_df[features])

# Prepare submission DataFrame
submission = test[['Item_Identifier', 'Outlet_Identifier']].copy()
submission['Item_Outlet_Sales'] = test_preds


submission['Item_Outlet_Sales'].clip(lower=0, inplace=True)
# Save to CSV
submission.to_csv('submission_xgboost.csv', index=False)

print("✅ Submission file 'submission_xgboost.csv' saved.")


✅ Submission file 'submission_xgboost.csv' saved.


In [ ]:
### ANN

In [547]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import backend as K




In [548]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols),
        ('num', StandardScaler(), [col for col in X_train.columns if col not in categorical_cols])
    ]
)


In [549]:
X_test  = test_df[features]
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_val_scaled = scaler.transform(X_val)
# X_test_scaled = scaler.transform(X_test)


# Fit on training data
X_train_encoded = preprocessor.fit_transform(X_train)
X_val_encoded = preprocessor.transform(X_val)
X_test_encoded = preprocessor.transform(X_test)



# X_train_scaled = scaler.fit_transform(X_full_train)
# # X_val_scaled = scaler.transform(X_val)
# # X_test_scaled = scaler.transform(X_test)



In [550]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau



early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.8,        # reduce by half
    patience=10,        # if val_loss doesn’t improve for 5 epochs
    min_lr=1e-6,
    verbose=1
)

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

model = Sequential([
    Dense(64, input_dim=X_train_encoded.shape[1], activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    # BatchNormalization(),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer=Adam(learning_rate=0.001),    loss=rmse,     
    metrics=[rmse] )


d:\Anconda_new\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [553]:
# early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X_train_encoded, y_train,
    validation_data=(X_val_encoded, y_val),
    epochs=250,
    batch_size=64,
    # callbacks=[reduce_lr],
    verbose=1
)


Epoch 1/250


120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1121.9515 - rmse: 1121.9547 - val_loss: 1013.9398 - val_rmse: 1019.6498
Epoch 2/250
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1115.2325 - rmse: 1115.2349 - val_loss: 1013.8835 - val_rmse: 1019.5859
Epoch 3/250
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1099.7574 - rmse: 1099.7596 - val_loss: 1013.7935 - val_rmse: 1019.4826
Epoch 4/250
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1140.2422 - rmse: 1140.2428 - val_loss: 1013.6262 - val_rmse: 1019.3116
Epoch 5/250
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1107.0573 - rmse: 1107.0568 - val_loss: 1014.0966 - val_rmse: 1019.8278
Epoch 6/250
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1144.4458 - rmse: 1144.4426 - val_loss: 1013.5617 - val_rmse: 1019.2263
Epoch 7/250
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1103.2423 - rmse: 1103.2390 - val_loss: 1013.5959 - val_rmse: 1019.2758
Epoch 8/250
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1115.5511 - rmse: 11

KeyboardInterrupt: 

In [ ]:
# # early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# history = model.fit(
#     X_train_scaled, y_train,
#     validation_data=(X_val_scaled, y_val),
#     epochs=200,
#     batch_size=64,
#     callbacks=[reduce_lr],
#     verbose=1
# )


Epoch 1/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1108.2147 - rmse: 1108.1992 - val_loss: 1005.2528 - val_rmse: 1003.9106 - learning_rate: 1.0000e-06
Epoch 2/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1120.8899 - rmse: 1120.8779 - val_loss: 1005.3051 - val_rmse: 1003.9420 - learning_rate: 1.0000e-06
Epoch 3/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1135.3695 - rmse: 1135.3514 - val_loss: 1005.2919 - val_rmse: 1003.9293 - learning_rate: 1.0000e-06
Epoch 4/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1142.3370 - rmse: 1142.3334 - val_loss: 1005.2426 - val_rmse: 1003.8891 - learning_rate: 1.0000e-06
Epoch 5/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1129.5983 - rmse: 1129.6219 - val_loss: 1005.3969 - val_rmse: 1004.0538 - learning_rate: 1.0000e-06
Epoch 6/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1136.1104 - rmse: 1136.0922 - val_loss: 1005.3150 - val_rmse: 1003.9651 - learning_rate: 1.0000e-06
Epoch 7/200
107/107 ━━━━━━━━━━━━━━

In [552]:
test_preds_ann = model.predict(X_test_encoded).flatten()
submission_ann = pd.DataFrame({
    'Item_Identifier': test['Item_Identifier'],
    'Outlet_Identifier': test['Outlet_Identifier'],
    'Item_Outlet_Sales': test_preds_ann
})
submission_ann.to_csv("submission_ann.csv", index=False)



178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 967us/step
